In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

In [16]:
strategy_type = "timing"
strategy_name = "WMAStrategy"
output_file = os.path.join("output", strategy_type, strategy_name, "2004-01-01_2024-01-01.pkl")
with open(output_file, "rb") as f:
    all_results = pickle.load(f)

In [17]:
# calculate the average return
avg_annual_return = 0
avg_sharpe_ratio = 0
avg_annual_volatility = 0
avg_max_drawdown = 0

# level 0 keys
rolling_windows = all_results.keys()
# level 1 keys
tickers = all_results[list(rolling_windows)[0]].keys()

results_df_by_tickers = pd.DataFrame(columns=["Period", "ticker", "annual_return", "sharpe_ratio", "annual_volatility", "max_drawdown"])

for ticker in tickers:
    for window in rolling_windows:
        avg_annual_return += all_results[window][ticker]["annual_return"]
        avg_sharpe_ratio += all_results[window][ticker]["sharpe_ratio"]
        avg_annual_volatility += all_results[window][ticker]["annual_volatility"]
        avg_max_drawdown += all_results[window][ticker]["max_drawdown"]
        results_df_by_tickers = results_df_by_tickers._append({"Period": window, "ticker": ticker, "annual_return": all_results[window][ticker]["annual_return"], "sharpe_ratio": all_results[window][ticker]["sharpe_ratio"], "annual_volatility": all_results[window][ticker]["annual_volatility"], "max_drawdown": all_results[window][ticker]["max_drawdown"]}, ignore_index=True)

    avg_annual_return /= len(rolling_windows)
    avg_sharpe_ratio /= len(rolling_windows)
    avg_annual_volatility /= len(rolling_windows)
    avg_max_drawdown /= len(rolling_windows)
    
    results_df_by_tickers = results_df_by_tickers._append({"Period": "Average", "ticker": ticker, "annual_return": avg_annual_return, "sharpe_ratio": avg_sharpe_ratio, "annual_volatility": avg_annual_volatility, "max_drawdown": avg_max_drawdown}, ignore_index=True)

/var/folders/07/x2w_c3y1707gvzfy_69bydyc0000gn/T/ipykernel_9203/3159752964.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_by_tickers = results_df_by_tickers._append({"Period": window, "ticker": ticker, "annual_return": all_results[window][ticker]["annual_return"], "sharpe_ratio": all_results[window][ticker]["sharpe_ratio"], "annual_volatility": all_results[window][ticker]["annual_volatility"], "max_drawdown": all_results[window][ticker]["max_drawdown"]}, ignore_index=True)


In [18]:
results_df_by_tickers.to_csv(os.path.join("output", strategy_type, strategy_name, "results.csv"), index=False)

In [33]:
strategy_type = "selection"
strategy_name = "TrendFollowingStrategy"
output_file = os.path.join("output", strategy_type, strategy_name, "2004-01-01_2024-01-01.pkl")
with open(output_file, "rb") as f:
    all_results = pickle.load(f)

In [34]:
avg_annual_return = 0
avg_sharpe_ratio = 0
avg_annual_volatility = 0
avg_max_drawdown = 0

# level 0 keys
rolling_windows = all_results.keys()

results_df = pd.DataFrame(columns=["Period", "annual_return", "sharpe_ratio", "annual_volatility", "max_drawdown"])

for window in rolling_windows:
    avg_annual_return += all_results[window]["annual_return"]
    avg_sharpe_ratio += all_results[window]["sharpe_ratio"]
    avg_annual_volatility += all_results[window]["annual_volatility"]
    avg_max_drawdown += all_results[window]["max_drawdown"]
    results_df = results_df._append({"Period": window, "annual_return": all_results[window]["annual_return"], "sharpe_ratio": all_results[window]["sharpe_ratio"], "annual_volatility": all_results[window]["annual_volatility"], "max_drawdown": all_results[window]["max_drawdown"]}, ignore_index=True)

avg_annual_return /= len(rolling_windows)
avg_sharpe_ratio /= len(rolling_windows)
avg_annual_volatility /= len(rolling_windows)
avg_max_drawdown /= len(rolling_windows)

results_df = results_df._append({"Period": "Average", "annual_return": avg_annual_return, "sharpe_ratio": avg_sharpe_ratio, "annual_volatility": avg_annual_volatility, "max_drawdown": avg_max_drawdown}, ignore_index=True)

/var/folders/07/x2w_c3y1707gvzfy_69bydyc0000gn/T/ipykernel_90602/2373360147.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = results_df._append({"Period": window, "annual_return": all_results[window]["annual_return"], "sharpe_ratio": all_results[window]["sharpe_ratio"], "annual_volatility": all_results[window]["annual_volatility"], "max_drawdown": all_results[window]["max_drawdown"]}, ignore_index=True)


In [35]:
results_df.to_csv(os.path.join("output", strategy_type, strategy_name, "results.csv"), index=False)